In [1]:
import sqlite3

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
db = sqlite3.connect('../raw_db.db')

dct = pd.read_csv('../data/dict.csv').drop('Unnamed: 0', axis=1)

dct.columns = [
        'category1_id'
        , 'category2_id'
        , 'category3_id'
        , 'category1_name'
        , 'category2_name'
        , 'category3_name'
        , 'category'
]

dct.head()

,category1_id,category2_id,category3_id,category1_name,category2_name,category3_name,category
0,649,651,1043,Игры,Интеллектуальные игры,Брейн-ринг,Для ума
1,649,651,1040,Игры,Интеллектуальные игры,Викторины,Для ума
2,649,651,1042,Игры,Интеллектуальные игры,Иные интеллектуальные игры,Для ума
3,649,651,1041,Игры,Интеллектуальные игры,Квест,Для ума
4,649,650,516,Игры,Настольные игры,Иные настольные игры,Для ума


In [3]:
db2 = sqlite3.connect('../app/app.db')
cursor = db2.cursor()

In [10]:
cursor.execute("""
CREATE TABLE categories (
    category1_id INTEGER, 
    category2_id INTEGER, 
    category3_id INTEGER, 
    category1_name TEXT, 
    category2_name TEXT, 
    category3_name TEXT, 
    category TEXT,
    PRIMARY KEY (category3_id)
)
""")
db2.commit()

In [11]:
dct.to_sql('categories', if_exists='append', con=db2, index=False)

484

In [15]:
attend = pd.read_sql("""
SELECT 
    attend.group_id as group_id
    , category3_id
    , user_id
    , min(event_date) as start_date
    , max(event_date) as finish_date
    , count(user_id) as n_visits
FROM attend
LEFT JOIN groups ON attend.group_id = groups.group_id
WHERE category3_id IS NOT NULL
GROUP BY attend.group_id, category3_id, user_id
""", con=db)

In [18]:
attend.head()

,group_id,category3_id,user_id,start_date,finish_date,n_visits
0,801346549,131,101370217,2022-12-05,2022-12-28,8
1,801346549,131,101375025,2022-09-14,2022-11-23,21
2,801346549,131,101392971,2022-12-28,2022-12-28,1
3,801346550,120,101352023,2022-04-04,2022-12-26,64
4,801346550,120,101354499,2022-04-18,2022-12-28,64


In [19]:
cursor.execute("""
CREATE TABLE attend_history (
    group_id INTEGER, 
    category3_id INTEGER, 
    user_id INTEGER, 
    start_date TEXT, 
    finish_date TEXT, 
    n_visits INTEGER, 
    PRIMARY KEY (user_id, group_id)
)
""")
db2.commit()

In [20]:
attend.to_sql('attend_history', if_exists='append', con=db2, index=False)

600185

In [23]:
group_locations = pd.read_sql("""
SELECT DISTINCT group_id, district_id as g_district, zone_id as g_zone, coalesce(code, 0) as g_postal_code
FROM group_locations
""", con=db)

In [28]:
cursor.execute("""
CREATE TABLE group_locations (
    group_id INTEGER, 
    g_district INTEGER, 
    g_zone INTEGER, 
    g_postal_code INTEGER, 
    PRIMARY KEY (group_id, g_district, g_zone, g_postal_code)
)
""")
db2.commit()

In [29]:
group_locations.to_sql('group_locations', if_exists='append', con=db2, index=False)

19624

In [47]:
group_timetable = pd.read_sql("""
SELECT group_id 
    , start_date
    , finish_date
    , weekday
    , start_time
FROM group_timetable
WHERE finish_date >= '2023-03-01'
""", con=db)

In [48]:
group_timetable = group_timetable.pivot_table(
    index=['group_id', 'start_date', 'finish_date']
    , columns='weekday'
    , values='start_time'
).fillna(0).astype('int8').reset_index().sort_values(by=['group_id', 'start_date', 'finish_date'])

In [53]:
group_timetable.columns = [
    'group_id', 'start_date', 'finish_date', 
    'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

In [50]:
cursor.execute("""
CREATE TABLE group_timetable (
    group_id INTEGER, 
    start_date TEXT, 
    finish_date TEXT, 
    monday INTEGER, 
    tuesday INTEGER, 
    wednesday INTEGER, 
    thursday INTEGER, 
    friday INTEGER, 
    saturday INTEGER, 
    sunday INTEGER, 
    PRIMARY KEY (group_id, start_date, finish_date)
)
""")
db2.commit()

In [54]:
group_timetable.to_sql('group_timetable', if_exists='append', con=db2, index=False)

15065

In [59]:
users = pd.read_sql_query("""
    SELECT 
        user_id
        , is_woman 
        , CAST((julianday('2023-03-01') - julianday(date_of_birth)) / 365 / 10 AS INT) * 10 as age_group
        , coalesce(postal_code, 0) as u_postal
    FROM users
""", con=db)

In [57]:
cursor.execute("""
CREATE TABLE users (
    user_id INTEGER, 
    is_woman INTEGER, 
    age_group INTEGER, 
    u_postal INTEGER, 
    PRIMARY KEY (user_id)
)
""")
db2.commit()

In [60]:
users.to_sql('users', if_exists='append', con=db2, index=False)

52334

In [66]:
groups = pd.read_sql("""
SELECT 
    groups.group_id 
    , category3_id
    , is_online
    , is_available
    , is_dead
    , ends_soon
    , address
FROM groups
LEFT JOIN (
    SELECT 
        group_id
        , max(finish_date) > '2023-03-01' AND sum(start_date < '2023-05-01') > 0 as is_available 
        , max(finish_date) > '2023-03-01' AND max(finish_date) < '2023-03-15' as ends_soon
        , max(finish_date) < '2023-03-01' as is_dead
    FROM group_timetable
    GROUP BY group_id
) ava ON groups.group_id = ava.group_id
""", con=db).fillna(0)

In [65]:
cursor.execute("""
CREATE TABLE groups (
    group_id INTEGER, 
    category3_id INTEGER, 
    is_online INTEGER, 
    is_available INTEGER, 
    is_dead INTEGER, 
    ends_soon INTEGER, 
    address TEXT,
    PRIMARY KEY (group_id)
)
""")
db2.commit()

In [69]:
groups.to_sql('groups', if_exists='append', con=db2, index=False)

27001

## Веса по категориям

In [70]:
group_user_stats = pd.read_sql("""
    SELECT 
        group_id
        , attend.user_id as user_id
        , count(event_id) as n
        , postal_code
        , max(is_woman) as is_woman
        , (julianday('2023-03-01') - julianday(date_of_birth)) / 365 as age
    FROM attend
    LEFT JOIN users u ON u.user_id = attend.user_id
    GROUP BY group_id, attend.user_id
""", con=db)

group_info = pd.read_sql("""
    SELECT group_id, category1_id, category2_id, category3_id, is_online, is_mobile
    FROM groups
""", con=db).dropna().astype(int) # (0)

group_user_stats = group_user_stats.merge(group_info)

In [71]:
cat3_user = group_user_stats.groupby(['category3_id', 'user_id'], as_index=False).agg({'n': 'sum'})

In [72]:
cat3_user['category3_id'].nunique()

293

In [101]:
from collections import defaultdict
from itertools import combinations
from math import log2

In [102]:
pairwise_dict = defaultdict(lambda: defaultdict(int))
uni_dict = defaultdict(int)

for row in tqdm(cat3_user.groupby('user_id').agg({'category3_id': list})['category3_id'].values):
    for (i, j) in combinations(row, 2):
        pairwise_dict[i][j] += 1
        pairwise_dict[j][i] += 1
        uni_dict[i] += 1
        uni_dict[j] += 1

  0%|          | 0/50904 [00:00<?, ?it/s]

In [103]:
result_dict = defaultdict(dict)
for i in pairwise_dict:
    for j in pairwise_dict[i]:
#         result_dict[i][j] = 50 + log2(pairwise_dict[i][j] * 2 / (uni_dict[i] + uni_dict[j]))
        result_dict[i][j] = np.log2(1 + pairwise_dict[i][j] / uni_dict[i] * pairwise_dict[j][i] / uni_dict[j] * 1000000)

In [104]:
result_flat = []

for i in result_dict:
    for j in result_dict[i]:
        result_flat.append((i, j, round(result_dict[i][j], 5)))

In [105]:
cat3_similarity = pd.DataFrame(result_flat, columns=['category_in', 'category_out', 'similarity'])#
cat3_similarity = cat3_similarity.sort_values(by=['category_in', 'similarity'], ascending=[True, False])
cat3_similarity['rank_cat'] = cat3_similarity.groupby(['category_in'])['similarity'].transform('rank', ascending=False)
cat3_similarity = cat3_similarity.sort_values(by=['category_in', 'rank_cat'], ascending=True)

In [106]:
cat3_similarity.head()

,category_in,category_out,similarity,rank_cat
29241,102,492,7.98489,1.0
29270,102,151,7.82190,2.0
29307,102,458,7.72980,3.0
29271,102,470,6.82812,4.0
29324,102,465,6.54934,5.0


In [107]:
cursor.execute("""
CREATE TABLE model_categories (
    category_in INTEGER, 
    category_out INTEGER, 
    similarity REAL, 
    rank_cat INTEGER, 
    PRIMARY KEY (category_in, category_out)
)
""")
db2.commit()

In [108]:
cat3_similarity['rank_cat'] = cat3_similarity['rank_cat'].astype(int)

In [109]:
cat3_similarity.to_sql('model_categories', if_exists='append', con=db2, index=False)

44862

In [157]:
user_age_group_raw = pd.read_sql("""
    SELECT 
        DISTINCT 
        users.user_id
        , round((julianday('2023-03-01') - julianday(date_of_birth)) / 365) as person_age
        , is_woman
        , category1_id
        , category2_id
        , category3_id
        , 1 as i
    FROM attend
    INNER JOIN users ON attend.user_id = users.user_id
    INNER JOIN groups ON attend.group_id = groups.group_id
""", con=db)
user_age_group_raw = user_age_group_raw.dropna().astype(int)
user_age_group_raw.shape

(279784, 7)

In [158]:
user_age_group = user_age_group_raw.pivot_table(index=['user_id', 'person_age', 'is_woman'], columns='category3_id', values='i').fillna(0).astype(int)
user_age_group = user_age_group.reset_index()
user_age_group['age_group'] = (user_age_group['person_age'] // 10) * 10

user_age_group_mean = user_age_group.drop(['user_id', 'person_age'], axis=1).groupby(['is_woman', 'age_group']).agg('mean')

names = {i: (i, c, x, y, z) for i, c, x, y, z in dct[['category3_id', 'category', 'category1_name', 'category2_name', 'category3_name']].drop_duplicates().sort_values(by='category3_id').values}

user_age_group_mean.columns = [names.get(i) for i in user_age_group_mean.columns]
user_age_group_mean = user_age_group_mean.T

user_age_group_mean = user_age_group_mean.reset_index().melt(id_vars=['index'])
user_age_group_mean['index'] = user_age_group_mean['index'].apply(lambda x: x[0])
user_age_group_mean.columns = ['category3_id', 'is_woman', 'age_group', 'age_cat_prob']

user_age_group_mean.head()

,category3_id,is_woman,age_group,age_cat_prob
0,102,0,40,0.000000
1,104,0,40,0.285714
2,111,0,40,0.000000
3,112,0,40,0.000000
4,114,0,40,0.285714


In [159]:
user_age_group_mean = user_age_group_mean.sort_values(by='age_cat_prob', ascending=False)

In [160]:
user_age_group_mean['rank'] = user_age_group_mean.groupby(['age_group', 'is_woman'])['age_cat_prob'].transform('rank', ascending=False)

In [161]:
user_age_group_mean.columns = ['category_out', 'is_woman', 'age_group', 'age_cat_prob', 'rank_age']

In [162]:
user_age_group_mean[(user_age_group_mean['is_woman'] == 1) & (user_age_group_mean['age_group'] == 90)]

,category_out,is_woman,age_group,age_cat_prob,rank_age
3242,139,1,90,0.166667,1.0
3237,130,1,90,0.145833,2.0
3238,131,1,90,0.135417,5.0
3233,122,1,90,0.135417,5.0
3463,1421,1,90,0.135417,5.0
...,...,...,...,...,...
3363,496,1,90,0.000000,191.0
3362,495,1,90,0.000000,191.0
3361,494,1,90,0.000000,191.0
3360,492,1,90,0.000000,191.0


In [163]:
user_age_group_mean['rank_age'] = user_age_group_mean['rank_age'].astype(int)

In [169]:
cursor.execute("""
CREATE TABLE model_age (
    category_out INTEGER, 
    is_woman INTEGER, 
    age_group INTEGER, 
    age_cat_prob REAL, 
    rank_age INTEGER, 
    PRIMARY KEY (category_out, is_woman, age_group)
)
""")
db2.commit()

In [170]:
user_age_group_mean.to_sql('model_age', if_exists='append', con=db2, index=False)

3516

In [117]:
postal_to_zd = pd.read_sql("""
    SELECT district_id, zone_id, code, count(group_id) as n
    FROM group_locations
    WHERE code IS NOT NULL
    GROUP BY district_id, zone_id, code
    ORDER BY n DESC
""", con=db, dtype={'district_id': 'int16', 'zone_id': 'int16', 'code': 'int32', 'n': 'int16'}).drop_duplicates(subset=['code'], keep='first')

In [119]:
cursor.execute("""
CREATE TABLE postal_map (
    code INTEGER, 
    zone_id INTEGER, 
    district_id INTEGER, 
    PRIMARY KEY (code)
)
""")
db2.commit()

In [121]:
postal_to_zd.drop('n', axis=1).to_sql('postal_map', if_exists='append', con=db2, index=False)

356

In [124]:
neighbors = pd.read_sql("""
    SELECT group_id, postal_code, count() - 1 as n_neighbors
    FROM (
        SELECT DISTINCT 
            group_id
            , users.postal_code
            , attend.user_id
        FROM attend
        LEFT JOIN users ON attend.user_id = users.user_id
        WHERE postal_code IS NOT NULL 
                AND event_date >= '2022-12-01'
    )
    GROUP BY group_id, postal_code
    HAVING n_neighbors > 1
""", con=db)
neighbors.shape

(18739, 3)

In [125]:
cursor.execute("""
CREATE TABLE model_neighbors (
    group_id INTEGER, 
    postal_code INTEGER, 
    n_neighbors INTEGER, 
    PRIMARY KEY (group_id, postal_code)
)
""")
db2.commit()

In [126]:
neighbors.to_sql('model_neighbors', if_exists='append', con=db2, index=False)

18739

In [21]:
questionnaire = pd.read_csv("/home/dkbrz/Downloads/categories - Лист2.csv")

In [22]:
questionnaire = questionnaire.melt(id_vars='category3_id').dropna()
questionnaire.columns = ['category3_id', 'feature', 'i']
questionnaire['i'] = questionnaire['i'].astype(int)

In [23]:
questionnaire.to_sql('questionnaire', con=db2, if_exists='replace')

185